In [1]:
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# change the max. print value
np.set_printoptions(threshold=1000000, suppress=True)

In [2]:
# 固定亂數種子
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [3]:
# 數據預處理
transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),# 對圖像進行隨機裁減
#     transforms.RandomRotation(20),# 對圖像進行隨機旋轉
#     transforms.RandomHorizontalFlip(p=0.5),# 對圖像進行隨機水平翻轉
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()# 變成tensor格式
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225])
]) # 數據增強

# 讀取數據
root = "./data/custom/MNIST"
train_data = datasets.ImageFolder(root + "/train",transform)
test_data = datasets.ImageFolder(root + "/test",transform)

In [4]:
batch_size = 256
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
feature_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)

In [5]:
classes = train_data.classes
classes_index = train_data.class_to_idx
print(classes)
print(classes_index)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [6]:
# 測試CUDA能否使用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
# 轉換成CUDA能讀的格式
# generate data in parallel with PyTorch
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 1, 28, 28])
Shape of y: torch.Size([256]) torch.int64


In [8]:
# 定義網路結構
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()# 初始化
        # nn.Conv2d(1,32,5,1,2): 通道數，輸出，捲積，步長，在圖片旁邊補0(幾圈)，激勵函數，最大池化2*2
        self.features = nn.Sequential(
            nn.Conv2d(1,32,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
    
        #64個特徵圖大小7*7 => 輸出500個特徵圖
        self.classifier = nn.Sequential(
            nn.Linear(3136,500),
            nn.Sigmoid(),
            nn.Linear(500,10),
            nn.Softmax(dim=1)
        ) 
 
        
    def forward(self,x):
         # torch.Size([64, 1, 28, 28]) # 捲積中需要傳入4維 批次大小 圖像通道數 圖片大小
        x = self.features(x)
        # torch.Size([64, 1, 28, 28]) -> (64,784)
        x = x.view(x.size()[0],-1) # 4維變2維（在全連接層做計算只能2維）
        x = self.classifier(x)
        return x


In [9]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # Extract Net Feature Layers
        self.features = list(model.features)
        self.features = nn.Sequential(*self.features)
        # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
        # Extract the first part of fully-connected layer from Net
        self.fc = model.classifier[0]
    def forward(self, x):
        # It will take the input 'x' until it returns the feature vector called 'out'
        out = self.features(x)
        out = self.flatten(out)
        out = self.fc(out) 
        return out 

# Initialize the model
model = Net().to(device)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [10]:
model

Net(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3136, out_features=500, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=500, out_features=10, bias=True)
    (3): Softmax(dim=1)
  )
)

In [11]:
new_model

FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=3136, out_features=500, bias=True)
)

In [12]:
features = []
size = len(feature_dataloader.dataset)
model.train()
for batch, (X, y) in enumerate(feature_dataloader):
    X, y = X.to(device), y.to(device) # 讀圖片跟labels
    
    model.eval() # 測試模式，不做反向傳播
    with torch.no_grad(): # 不需要計算梯度
     # Compute prediction error
        feature = new_model(X)
        
    if batch % 100 == 0: # 顯示進度條
        current = batch * len(X)
        print(f"Feature Extractor: [{current:>5d}/{size:>5d}]")
            
    # Convert to NumPy Array, Reshape it, and save it to features variabl
    features.append(feature.cpu().detach().numpy())    

Feature Extractor: [    0/60000]
Feature Extractor: [25600/60000]
Feature Extractor: [51200/60000]


In [13]:
# Convert to NumPy Array
# 由於資料是不規則numpy因此會報錯，這裡因美觀我把它關掉
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
featuresdata = np.array(features)

In [14]:
# print(featuresdata[0],featuresdata[234])
print("Calculate: 60000 = 256 * 234 + 96")
print("total: ",len(featuresdata))
print('first batch: ',len(featuresdata[0]))
print('final batch: ',len(featuresdata[234]))

Calculate: 60000 = 256 * 234 + 96
total:  235
first batch:  256
final batch:  96


In [15]:
# 因為資料不平整，要用此方法去處理成一維numpy
import itertools
featureslist =list(itertools.chain(*featuresdata))
# featureslist

In [16]:
len(featureslist)

60000

In [17]:
# featureslist[0]

In [18]:
len(featureslist[0])

500

In [19]:
# 我嘗試包成def，但要如何回傳出features?，回傳能用此值嗎?

# def extractor(dataloader, model):
#     features = []
#     size = len(dataloader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(dataloader):
#         X, y = X.to(device), y.to(device)

#         # Compute prediction error
#         feature = model(X)
        
#         if batch % 100 == 0:
#             current = batch * len(X)
#             print(f"[{current:>5d}/{size:>5d}]")
            
#     # Convert to NumPy Array, Reshape it, and save it to features variabl
#     features.append(feature.cpu().detach().numpy().reshape(-1))    
#     return features

# extractor(train_dataloader, new_model)

In [20]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())
 
# def train(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     model.train()
#     for batch, (X, y) in enumerate(dataloader):
#         X, y = X.to(device), y.to(device)
 
#         # Compute prediction error
#         pred = model(X)
#         loss = loss_fn(pred, y)
 
#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
 
#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# def test(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0
#     with torch.no_grad():
#         for X, y in dataloader:
#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [21]:
# epochs = 10
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(train_dataloader, model, loss_fn, optimizer)
#     test(test_dataloader, model, loss_fn)
# print("Done!")